# About

Extract specific parts from JSON documents and store them as columns in a CSV/TSV for further data analysis.

# Prerequesites

* A running mongodb instance which holds the JSON documents (see docker container).
* Python libraries:
    * pymongo - python bindings for mongodb.
    * jsonpath_rw - path expressions for matching parts of a JSON document.
* A configuration file (config.yaml) with setting for
    * mongodb instance and collection name
    * JSON document field selection

In [ ]:
from pymongo import MongoClient
from jsonpath_rw import jsonpath, parse
import json
import yaml
import pandas as pd
import re

## Prepare configuration

In [ ]:
config_file = 'config.yaml'

In [ ]:
with open(config_file) as yaml_file:
    cfg = yaml.load(yaml_file)

mongo_db = cfg['mongo']

json_parts = cfg['json-doc']['match_list']

## Prepare database connection

In [ ]:
client = MongoClient(mongo_db['url'])

db = client[mongo_db['database']][mongo_db['collection']]

print "%d entries in database." % db.find().count()

## Process all documents

Define function to clean text data such that it can be saved as one line.

In [ ]:
def clean(data):
    """Replace new line and tab; remove leading and trailing spaces."""
    return re.sub('[\t\n\r]', ' ', data).strip()

In [ ]:
records = [tuple(clean(data[field]) for field in json_parts) for data in db.find()]

df = pd.DataFrame.from_records(records, columns=json_parts)

## Save documents

In [ ]:
output_file = "../data/" + "_".join(json_parts) + ".tsv"

df.to_csv(output_file, sep="\t", index=False, quoting=3, encoding="utf-8")